# 言語モデルに、外部データを取り込む

用意されている言語モデルは、ある日時までのデータで学習済みですが、今日出た最新の内容を学習しているわけではありません。また、利用者が保持している固有のデータを学習しているわけではありません。

LLMを使ったアプリケーションの大半は、モデルの学習セットには含まれないユーザ固有のデータが必要になります。
これを実現する主な方法は、RAG (Retrieval Augmented Generation) です。これは、外部のデータを取得して、プロンプト生成の際にこのデータを埋め込んでLLMに渡します。

LangChainには、RAGを実現するためのツールが備わっています。

## 準備

In [1]:
## 利用するベースモデルのライブラリ (AWS SDK (boto3)) も別途インストールする
!pip install langchain==0.0.331 boto3 botocore awscli tiktoken

## LangChain Retrieval

RAG（Retrieval Augmented Generation）を実現するために必要なツールをまとめた、LangChainモジュールです。

![Retrieval](https://python.langchain.com/assets/images/data_connection-c42d68c3d092b85f50d08d4cc171fc25.jpg)

https://python.langchain.com/docs/modules/data_connection/

RAGを実現するための手順を簡単にみてみましょう。

### ドキュメントローダ (Load)

様々なソースからドキュメントをロードします。
LangChainは、100以上の異なるドキュメントローダを提供しています。
あらゆる種類のドキュメント（HTML／PDF／コード／等）をあらゆる場所（Amazon S3バケット／Webサイト／等）から読み込む機能が提供されています。

### ドキュメント変換 (Transform)

ドキュメント検索の重要なことは、ドキュメントの関連する部分のみをフェッチすることになります。そのために、検索用に最適化するための変換ステップが含まれます。
例えば、大きな文章を小さな文章に分割(chunking)することです。
LangChainは、これを行うためのアルゴリズムと、特定のドキュメントタイプ(コード／markdown／等)に最適化されたロジックを提供しています。

### テキスト埋め込み (Embed)

さらにドキュメント検索に重要なことは、テキストの埋め込み(Embedding)を作成することです。
Embeddingは、ざっくりに言うとテキスト文字列を数値ベクトルに置き換えます。
これをすることで、テキストの意味(semantec meaning)を捕らえ、類似した他のテキストを素早く効率的に検索することを可能にします。
LangChainは、25以上の異なるEmbedding Providerやメソッドとの統合を提供します。
LangChainは標準的なインターフェイスを提供し、簡単にモデル間の切り替えを可能にします。

### ベクター保存 (Vector Store)

テキスト文字列をEmbeddingすると、数値ベクトルに変換されます。
Embeddingで得られた値を効率よく保存・検索することを目的としたデータベースが必要になってきます。
LangChainは、50以上の異なるベクターストアとの統合を提供します。
LangChainは標準的なインターフェイスを公開しており、ベクターストアを簡単に交換することができます。

### 検索 (Retrieve)

ベクターデータがデータベースに格納されたら、それを取り出す必要があります。
LangChainは様々な検索アルゴリズムをサポートしています。
単純なセマンティック検索など、簡単に始められる基本的な方法をサポートしています。
しかし、パフォーマンスを向上させるために、これに加えて様々なアルゴリズムを追加しています。
以下がその例です：

- 親ドキュメント検索
    - 親ドキュメントごとに複数の埋め込みを作成することができ、より小さなチャンクを検索しつつ、より大きなコンテキストを返すことができます。
- セルフクエリ検索
    - ユーザーからの質問には、意味的なものだけでなく、メタデータフィルタとして表現するのが最適なロジックが含まれていることがよくあります。セルフクエリでは、クエリ内に存在する他のメタデータフィルタから、クエリのセマンティック部分を解析することができます。
- アンサンブル検索
    - 複数の異なるソースから、あるいは複数の異なるアルゴリズムを使って文書を取得したい場合があります。アンサンブル・リトリーバを使えば、これを簡単に行うことができます。


## ドキュメントローダ (Load)

シンプルな例を挙げます。ファイル内にあるテキストを読み込んで、1つのドキュメントオブジェクトとして配置します。


```python
from langchain.document_loaders import TextLoader

loader = TextLoader("./index.md")
loader.load()
```

今回はせっかくなので、PDFドキュメントを読み込んでみます。
試しに比較的大きめで、外部公開しても問題ないPDFドキュメントを格納しましょう。

**格納先**:

- SageMaker Studio
    - 左側のエクスプローラ画面を開いて、アップロードしましょう。
- SageMaker Studio Lab
    - 左側のエクスプローラ画面を開いて、アップロードしましょう。
- Google Coraboratory
    - Google Driveをマウントして、Google Driveのマイドライブにアップロードしましょう。


In [2]:
## PDFの読み込みに、今回はpypdfを使ってみます。
!pip install pypdf

In [3]:
## 読み込みたいドキュメントを指定
## サンプルとして、外務省が公開しているSDGsの基礎資料のPDFを読み込ませてみる。
## https://www.mofa.go.jp/mofaj/gaiko/oda/sdgs/about/index.html

pdfdoc = './doc/sdgs_gaiyou_202310.pdf'


In [4]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(pdfdoc)
pages = loader.load_and_split()

In [5]:
pages[0]

Document(page_content='持続可能な開発目標（ SDGs）達成に\n向けて日本が果たす役割\n1ＳＤＧｓを通じて、豊かで活力ある未来を創る\n令和5年10月\n外務省国際協力局 地球規模課題総括課', metadata={'source': './doc/sdgs_gaiyou_202310.pdf', 'page': 0})

In [6]:
pages[1]

Document(page_content='持続可能な開発目標（ SDGs）\n◼2015年9月の国連サミットで全会一致で採択。 「誰一人取り残さない」持続可能で多様性と包\n摂性のある社会 の実現のため、 2030年を年限とする 17の国際目標。 （その下に、 169のターゲット、\n231の指標が決められている。）\n先進国を含め、 全ての国が行動 普遍性\n人間の安全保障の理念を反映し\n「誰一人取り残さない 」 包摂性\n全てのステークホルダーが役割を 参画型\n社会・経済・環境に 統合的に取り組む 統合性\n定期的にフォローアップ 透明性\n\uf0752001年に国連で専門家間の議論を経て策定 。2000年に採択された 「国連ミレニアム宣言 」と、\n1990年代の主要な国際会議で採択された国際開発目標を統合したもの 。\n\uf075開発途上国向けの開発目標 として、 2015年を期限とする 8つの目標を設定。\n（①貧困・飢餓、②初等教育、③女性、④乳幼児、⑤妊産婦、⑥疾病、⑦環境、⑧連帯）前身：ミレニアム開発目標（ Millennium Development Goals: MDGs ）\n✓MDGsは一定の成果を達成。一方で、未達成の課題も残された。\n○極度の貧困半減（目標①）や HIV・マラリア対策（同⑥）等を達成。\n×乳幼児や妊産婦の死亡率削減（同④、⑤）は未達成。サブサハラアフリカ等で達成に遅れ。環境\n人権\n平和\n2', metadata={'source': './doc/sdgs_gaiyou_202310.pdf', 'page': 1})

ちなみに、今回はLangChainのドキュメントローダを利用しましたが、ファイル種別毎に用意されたライブラリを直接利用してテキスト抽出を行っても構いません。
個別のライブラリの方が多機能な部分があるため、そのような機能を使って抽出したい場合は、個別のライブラリを使う方が良いでしょう。


## ドキュメント変換 (Transform)

読み込んだドキュメントのテキストを元に、いくつかのチャンク(chunk)に分割します。
ここでは、LangChainの `RecursiveCharacterTextSplitter` を使います。

このテキスト分割ツールは、一般的なテキストに対して推奨されています。

今回は文字列長を元にチャンクを分割します。

ちなみに、トークン長を元にチャンクを分割したい場合は、OpenAIであれば、
 `from_tiktoken_encoder()` を使って、トークン長を元にチャンクを分割すると良いでしょう。
トークン長の推定は、OpenAIが公開している `tiktoken` というライブラリを用います。

https://github.com/openai/tiktoken


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#     ## 適切なチャンクサイズは、ドキュメントによって異なるため、調整は必要
#     ## 大きすぎると、回答時にいろんな箇所の情報が参照できない (トークン長に限りがあるため)
#     ## 小さすぎると、回答の文脈がおかしくなる (途中で文章切れてしまうため)
#     chunk_size = 200,
#     chunk_overlap  = 0,
# )

text_splitter = RecursiveCharacterTextSplitter(
    ## 適切なチャンクサイズは、ドキュメントによって異なるため、調整は必要
    ## 大きすぎると、回答時にいろんな箇所の情報が参照できない (トークン長に限りがあるため)
    ## 小さすぎると、回答の文脈がおかしくなる (途中で文章切れてしまうため)
    chunk_size = 200,  ## 通常はテキストサイズ
    chunk_overlap  = 10,
)

In [8]:
## pageの区切りを '\n\n' に変換して、一つの文字列にしてchunkを生成
chunks = text_splitter.split_text('\n\n'.join([page.page_content for page in pages]))

In [9]:
len(chunks)

33

In [10]:
## 最初の5つ分のchunkを表示
chunks[:5]

['持続可能な開発目標（ SDGs）達成に\n向けて日本が果たす役割\n1ＳＤＧｓを通じて、豊かで活力ある未来を創る\n令和5年10月\n外務省国際協力局 地球規模課題総括課',
 '持続可能な開発目標（ SDGs）\n◼2015年9月の国連サミットで全会一致で採択。 「誰一人取り残さない」持続可能で多様性と包\n摂性のある社会 の実現のため、 2030年を年限とする 17の国際目標。 （その下に、 169のターゲット、\n231の指標が決められている。）\n先進国を含め、 全ての国が行動 普遍性\n人間の安全保障の理念を反映し\n「誰一人取り残さない 」 包摂性',
 '全てのステークホルダーが役割を 参画型\n社会・経済・環境に 統合的に取り組む 統合性\n定期的にフォローアップ 透明性\n\uf0752001年に国連で専門家間の議論を経て策定 。2000年に採択された 「国連ミレニアム宣言 」と、\n1990年代の主要な国際会議で採択された国際開発目標を統合したもの 。\n\uf075開発途上国向けの開発目標 として、 2015年を期限とする 8つの目標を設定。',
 '（①貧困・飢餓、②初等教育、③女性、④乳幼児、⑤妊産婦、⑥疾病、⑦環境、⑧連帯）前身：ミレニアム開発目標（ Millennium Development Goals: MDGs ）\n✓MDGsは一定の成果を達成。一方で、未達成の課題も残された。\n○極度の貧困半減（目標①）や HIV・マラリア対策（同⑥）等を達成。',
 '×乳幼児や妊産婦の死亡率削減（同④、⑤）は未達成。サブサハラアフリカ等で達成に遅れ。環境\n人権\n平和\n2']

## テキスト埋め込み (Embedding)

Embedding可能なサービスは様々あります。
今回は、Bedrock上にAWSが提供しているEmbeddings APIサービスをLangChainから使います。

実際には、この後のベクター保存の際にEmbeddingを兼ねて保存されますが、Embedding自体の振る舞いをここでは見てみます。

In [11]:
import os
import boto3
from langchain.embeddings import BedrockEmbeddings

bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2'
)

embeddings_model = BedrockEmbeddings(
    client=bedrock_runtime,
    model_id='amazon.titan-embed-text-v1'
)

embeddings = embeddings_model.embed_documents(chunks)

In [12]:
len(embeddings), len(embeddings[0])

(33, 1536)

In [13]:
## Embeddingした結果、最初のchunkの最初から5つの数値を表示
embeddings[0][:5]

[1.3046875, 0.09277344, 0.26171875, 0.2890625, 0.78515625]

## ベクター保存 (Store)

Embeddingデータの保存と、ベクトル検索の実行を行います。
検索の際は、問い合わせクエリとなる文字列をEmbeddingして、保存されているデータとの類似度で検索をかけます。

![Vector Store](https://python.langchain.com/assets/images/vector_stores-9dc1ecb68c4cb446df110764c9cc07e0.jpg)

https://python.langchain.com/docs/modules/data_connection/vectorstores/

ベクター保存可能なサービスは、ここ最近急速に増えています。

今回は、SageMaker Studio LabやColaboratory上でも動作可能なローカルDBを利用します。



In [14]:
## ベクターストアとして、Chromaを利用 (それ以外のライブラリは依存関係解消のため)
## 0.4.16だと、LangChainからの呼び出しでエラーになるので、一旦0.4.15に固定
## https://github.com/langchain-ai/langchain/issues/13051

!pip install chromadb==0.4.15 kaleido python-multipart "typing-extensions>4.7.0" # "typing-extensions<4.6.0"

In [15]:
from langchain.vectorstores import Chroma

## データをベクターデータとして保存
db = Chroma.from_texts(texts=chunks, embedding=embeddings_model)

In [16]:
## 試しに検索をかけてみる　(文字列版)
query = "日本における、SDGsアクションプラン2023は何ですか？"
docs = db.similarity_search(query)
print(docs[0].page_content)

•SDGサミットや持続可能な開発のための国連ハイレベル政治フォーラム（ HLPF）、日メコン SDGsフォー
ラム等の議論に積極的に貢献。SDGsアクションプラン 2023：重点事項②
6


In [17]:
## 試しに検索をかけてみる　(ベクター版)
query = "日本における、SDGsアクションプラン2023は何ですか？"
embedding_vector = embeddings_model.embed_query(query)  ## 問合せクエリ文字列をベクター化
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

•SDGサミットや持続可能な開発のための国連ハイレベル政治フォーラム（ HLPF）、日メコン SDGsフォー
ラム等の議論に積極的に貢献。SDGsアクションプラン 2023：重点事項②
6


## 検索 (Retrieve)

実際の検索には、LangChain Retrieverを利用します。
LangChain Retrieverは、構造化されていないクエリを指定するとドキュメントを返すインタフェースになっています。ベクターストアはRetrieverのバックボーンとして利用します。

Retrieverは、LangChain Expression Language (LCEL) の基本構成要素であるRunnableインタフェースを実装しています。そのため、LCELでチェインを構成している場合、非常に便利です。
Runnableインタフェースは、invoke, ainvoke, stream, astream, batch, abatch, astream_log 呼び出しをサポートしています。

In [18]:
## Retrieverインタフェース
retriever = db.as_retriever()

In [19]:
## 検索
retrieved_docs = retriever.invoke(
    "日本における、SDGsアクションプラン2023は何ですか？"
)
print(retrieved_docs[0].page_content)

•SDGサミットや持続可能な開発のための国連ハイレベル政治フォーラム（ HLPF）、日メコン SDGsフォー
ラム等の議論に積極的に貢献。SDGsアクションプラン 2023：重点事項②
6


ここまでで、RAGの事前準備ができました。

## ChatModelを使って、PDFの内容を質問しよう

ここからいよいよ、PDFの内容をプロンプトを含めた状態でChat Modelに質問を問い合わせてみましょう。


In [20]:
from langchain.chat_models import BedrockChat
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from operator import itemgetter
from langchain.chains import ConversationalRetrievalChain

## Setup Prompt Template
## 今回はデフォルトのテンプレートを使うので、カスタム設定しないでおく。

## Setup Memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

## Setup Chat Model
llm = BedrockChat(
    client=bedrock_runtime,
    model_id='anthropic.claude-v2',
    model_kwargs={
        "temperature": 0.7,
        ## "max_tokens_to_sample": 500
    }
)    

## Setup Retriever
retriever = db.as_retriever(
    search_type='similarity',  ## 検索タイプ: 'similarity' / 'similarity_score_threshold' / 'mmr' (Maximum Marginal Relevance)
    search_kwargs={"k": 5},  ## 検索取得数: default: 4
)

## Setup Chain
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    verbose=True,
)


In [21]:
## 会話開始
query = "日本における、SDGsアクションプラン2023は何ですか？"
res = qa({"question": query})
res



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
•SDGサミットや持続可能な開発のための国連ハイレベル政治フォーラム（ HLPF）、日メコン SDGsフォー
ラム等の議論に積極的に貢献。SDGsアクションプラン 2023：重点事項②
6

年から毎年、 ８つの優先課題に基づき、政府の施策のうちの重点項目を整理した「 SDGsアクションプラン」を策
定している。また、 SDGs達成に資する優れた取組を行う企業・団体等を 「ジャパン SDGsアワード」 を通じて表彰して
いる。
32023年9月
SDGサミット開催
2023年中（予定）
SDGs実施指針の改定2023年はSDGsの「中間年」。

現のため、 SDGsの達成に向けた取組を加速化する必要 がある。
•2023年には「SDGs実施指針」の改定 が見込まれている。新たな実施指針を、 2030年のSDGs
達成に向けた 本格的な行動の加速・拡大 に資するものとすべく取り組んでいく。
•2023年5月、日本は、自由、民主主義、人権、法の支配といった基本的価値を共有する Ｇ７の

2030年までに国内外において
SDGsを達成するための中長期的な国家戦略。
2019年9月に開催された SDGサミットと、
日本国内における SDGsの取組進展を踏まえて改定。
※SDGs推進本部では、 SDGs実施指針に基づき、 SDGsへの貢献を「見える化」することを目的として、 2017

「SDGsアクションプラン 2023」作成に当たっての基本的な考え方
•2023年はSDGsの「中間年」 。世界は歴史的な分水嶺に立ち、新たな挑戦に直面。新型コロナや
気候変動に加え、ロシアによるウクライナ侵略、食料やエネ

{'question': '日本における、SDGsアクションプラン2023は何ですか？',
 'chat_history': [HumanMessage(content='日本における、SDGsアクションプラン2023は何ですか？'),
  AIMessage(content=' SDGsアクションプラン2023は、日本政府が2023年にSDGs達成に向けて重点的に取り組む施策のリストです。 \n\n具体的には、\n\n- 2023年はSDGsの「中間年」と位置づけられ、SDGs達成に向けた取組を加速化する必要があるため、政府は毎年「SDGsアクションプラン」を策定しています。\n\n- 2023年版は8つの優先課題に基づいて重点項目を整理したものです。\n\n- 2023年9月にはSDGサミットが開催され、2023年中にはSDGs実施指針の改定が予定されています。 \n\n- 改定後の新しい実施指針は、2030年のSDGs達成に向けた本格的な行動の加速・')],
 'answer': ' SDGsアクションプラン2023は、日本政府が2023年にSDGs達成に向けて重点的に取り組む施策のリストです。 \n\n具体的には、\n\n- 2023年はSDGsの「中間年」と位置づけられ、SDGs達成に向けた取組を加速化する必要があるため、政府は毎年「SDGsアクションプラン」を策定しています。\n\n- 2023年版は8つの優先課題に基づいて重点項目を整理したものです。\n\n- 2023年9月にはSDGサミットが開催され、2023年中にはSDGs実施指針の改定が予定されています。 \n\n- 改定後の新しい実施指針は、2030年のSDGs達成に向けた本格的な行動の加速・'}

より詳細なRAGの手法が知りたい場合は、以下のサイト等を参考にしましょう。

- [Retrieval-augmented generation (RAG) | 🦜️🔗 Langchain](https://python.langchain.com/docs/use_cases/question_answering/)
- [Retrieval | 🦜️🔗 Langchain](https://python.langchain.com/docs/modules/data_connection/)

## RAGアプリケーションを作る際の検討事項

大量のドキュメントの中から回答を得るような、大規模なRAGアプリケーションを構築する場合、いくつか検討する事項があります。


### 欲しい情報がどのドキュメントにあるのか？

RAGでは、既存のドキュメントの内容をプロンプト内に入れることで、その情報を元に回答してくれます。
ドキュメントが大量にある場合、まず既存のドキュメントの検索をどのように行うかを検討する必要があります。

今回のサンプルコードでは、ローカルメモリ上に構築したベクターストアにドキュメントのEmbedding情報を格納し、検索することを行いました。ローカルメモリ上では情報の永続化ができないので、永続化できないといけません。
今回使用した「Chroma」は、永続化オプションがあります。
しかし、ローカルマシン上に格納されるので、負荷が高くなった時のスケールアウトが難しくなります。

実際のアプリケーションでは、ベクターストアはアプリケーションとは別の外部にあることが望ましいです。

例えば、PostgreSQLにpg_vectorという拡張モジュールがあり、これを使うことでPostgreSQL上でベクトル管理・検索が可能になります。Amazon RDSにも対応しています。

- 参考: [Amazon RDS for PostgreSQLがpgvectorモジュールに対応しベクトル検索できるようになりました | DevelopersIO](https://dev.classmethod.jp/articles/amazon-rds-postgresql-pgvector-embedding/)

また、近年ベクターストアサービスがたくさん出てきています。

Momento Vector Indexや、Cloudflare Vectorizeといった、Webアプリケーション等で利用しやすいサービスもあります。

- 参考: [サーバレスなVector Database、Momento Vector Indexの紹介 | DevelopersIO](https://dev.classmethod.jp/articles/momento-vector-index/)
- 参考: [Cloudflare Vectorize · Vectorize](https://developers.cloudflare.com/vectorize)

### ドキュメント検索サービスを使う方法

ベクターストア以外にドキュメント検索する方法としては、ドキュメント検索サービスを使う方法があります。
AWSであれば、Amazon Kendra、Google Cloudであれば、Enterprize Searchがあります。
AWSはAmazon KendraとOpenAIを活用したサンプルチャットアプリケーションを公開しています。

- 参考: [Amazon Kendra と OpenAI により最新の AWS ユーザーガイドに基づいて回答するチャットアプリケーションのサンプルを試してみた | DevelopersIO](https://dev.classmethod.jp/articles/using-amazon-kendra-langchain-extensions/)
- 参考: [【Google Cloud】Enterprise Searchで社内ドキュメントを検索してみた | DevelopersIO](https://dev.classmethod.jp/articles/try-google-cloud-enterprise-search/)

ドキュメント検索サービスだけ使っても、言語モデルからの応答は得られません。
この場合、ドキュメント検索サービスの検索結果の上位のドキュメントから、必要な情報を抽出して、言語モデルのプロンプトに入れて回答を得る必要があります。

この方法では、大量のドキュメントを事前にベクターストアに格納する必要がなく、引用元のドキュメントもわかるという利点があります。

### 参考構成

![AWS_RAG](https://devio2023-media.developers.io/wp-content/uploads/2023/10/Untitled.png)

https://dev.classmethod.jp/articles/implement-rag-with-aws-services/

![GCP_RAG](https://devio2023-media.developers.io/wp-content/uploads/2023/09/Picture3-1-640x203.png)

https://dev.classmethod.jp/articles/improve-work-efficiency-with-generateive-ai-chatbot-using-rag/

![GCP_Enterprize_RAG](https://devio2023-media.developers.io/wp-content/uploads/2023/09/Picture1-1-1536x944.png)

https://dev.classmethod.jp/articles/qa-with-google-cloud-enterprise-search-and-retrieve-read-compose-rag/
